# 🔥 AIFFEL DLthon - DKTC 한국어 위협 대화 분류
## Korean Dangerous Talk Classification

**전략: 탑다운 접근법**
- Level 1: 베이스라인 → 문제 발견
- Level 2: 최신 논문 기법으로 문제 해결
- Level 3: Ablation Study로 정량 검증

**적용 기법:**
| 기법 | 논문 | 역할 |
|---|---|---|
| KcELECTRA | Clark et al., 2020 | 한국어 구어체 특화 모델 |
| Focal Loss | Lin et al., 2017 | 클래스 불균형 해결 |
| R-Drop | Liang et al., 2021 | 소규모 데이터 과적합 방지 |
| LLM 합성 + 공개 데이터 | AugGPT (Dai et al., 2023) 영감 | 일반대화 클래스 구축 |

**학습 이력 활용:**
- Ex03: Ablation Study 방법론
- Ex06: 텍스트 전처리, 감성분석 파이프라인
- Ex07: SentencePiece 토크나이저
- Ex09: Transformer 구조 이해 → Fine-tuning

---
## 📦 STEP 0: 환경 설정

In [ ]:
# ═══ 0-1: 라이브러리 설치 ═══
!pip install -q transformers datasets accelerate sentencepiece
!pip install -q scikit-learn pandas matplotlib seaborn

In [ ]:
# ═══ 0-2: 외부 데이터 다운로드 ═══
# SmileStyle: 한국어 멀티턴 대화 (17개 스타일)
!wget -q https://raw.githubusercontent.com/smilegate-ai/korean_smile_style_dataset/main/smilestyle_dataset.tsv

# KakaoChatData: 실제 카카오톡 대화 73K개
!wget -q https://raw.githubusercontent.com/Ludobico/KakaoChatData/main/Dataset/ChatbotData.csv

print('✅ 외부 데이터 다운로드 완료')
!ls -la *.tsv *.csv 2>/dev/null || echo '파일 확인 필요'

In [ ]:
# ═══ 0-3: 공통 라이브러리 임포트 & 시드 고정 ═══
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re, os, random, warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    get_linear_schedule_with_warmup
)
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# GPU 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'🖥️ Device: {device}')
if torch.cuda.is_available():
    print(f'   GPU: {torch.cuda.get_device_name(0)}')

def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

set_seed(42)

# 한글 폰트 설정 (시각화용)
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# ═══ 0-4: 공통 상수 & 클래스 매핑 ═══
CLASS_MAP = {
    '협박 대화': 0,
    '갈취 대화': 1,
    '직장 내 괴롭힘 대화': 2,
    '기타 괴롭힘 대화': 3,
    '일반 대화': 4
}
CLASS_NAMES = list(CLASS_MAP.keys())
NUM_CLASSES = 5
MAX_LEN = 256
BATCH_SIZE = 16

print(f'클래스 매핑: {CLASS_MAP}')

---
## 📊 STEP 1: 데이터 로드 및 EDA

**[배운 것 활용]** Ex06 네이버 영화리뷰 감성분석에서 배운 데이터 분석 기법 적용
- 클래스 분포, 문장 길이, 빈출 단어 분석

In [ ]:
# ═══ 1-1: 데이터 로드 ═══
# ※ Colab에 train.csv, test.csv, submission.csv를 업로드하세요
# from google.colab import files
# uploaded = files.upload()

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
submission_df = pd.read_csv('submission.csv')

print(f'Train: {train_df.shape}')
print(f'Test:  {test_df.shape}')
print(f'\n클래스 분포:')
print(train_df['class'].value_counts())
print(f'\n⚠️ 일반 대화: 0개 → 합성데이터 필수!')

In [ ]:
# ═══ 1-2: EDA 시각화 ═══
train_df['conv_len'] = train_df['conversation'].str.len()
train_df['label'] = train_df['class'].map(CLASS_MAP)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 클래스 분포
train_df['class'].value_counts().plot(kind='bar', ax=axes[0],
    color=['#e74c3c','#f39c12','#3498db','#2ecc71'])
axes[0].set_title('Class Distribution')
axes[0].tick_params(axis='x', rotation=45)

# 대화 길이 분포
for cls in train_df['class'].unique():
    subset = train_df[train_df['class']==cls]['conv_len']
    axes[1].hist(subset, bins=30, alpha=0.5, label=cls)
axes[1].set_title('Conversation Length by Class')
axes[1].set_xlabel('Characters')
axes[1].legend(fontsize=7)

# 길이 boxplot
train_df.boxplot(column='conv_len', by='class', ax=axes[2])
axes[2].set_title('Length Boxplot')
axes[2].set_xlabel('')

plt.tight_layout()
plt.show()

print(f'\n길이 통계:')
print(train_df.groupby('class')['conv_len'].describe()[['mean','50%','max']])

In [ ]:
# ═══ 1-3: 클래스별 빈출 단어 (상위 10개) ═══
for cls in train_df['class'].unique():
    texts = ' '.join(train_df[train_df['class']==cls]['conversation'].tolist())
    words = [w for w in texts.split() if len(w) > 1]
    common = Counter(words).most_common(10)
    print(f'\n[{cls}]')
    print(f"  {', '.join([f'{w}({c})' for w, c in common])}")

---
## 🏭 STEP 2: 일반 대화 합성데이터 수집

**[최신 기법]** AugGPT (Dai et al., 2023) 영감 - LLM/공개데이터로 일반대화 구축

**[과제 규칙]** 일반대화 클래스는 합성데이터로 구성 (필수)

### 5개 데이터 소스:
| # | 소스 | 설명 | 목표 수 |
|---|---|---|---|
| 1 | SmileStyle | 한국어 멀티턴 대화 informal/chat 스타일 | 400개 |
| 2 | KakaoChatData | 실제 카카오톡 대화 | 300개 |
| 3 | kor_unsmile clean | 혐오 아닌 일반 구어체 | 200개 |
| 4 | NSMC 긍정 | 네이버 영화리뷰 일상 표현 | 100개 |
| 5 | 경계 케이스 ⭐ | 위협처럼 보이지만 일반 | 25개 |

In [ ]:
# ═══ 2-1: 소스별 데이터 수집 함수 ═══

def load_smilestyle(path='smilestyle_dataset.tsv', max_n=400):
    """SmileStyle에서 informal/chat 스타일 대화 추출"""
    try:
        df = pd.read_csv(path, sep='\t')
        convs = []
        for col in ['informal', 'chat', 'choding']:
            if col in df.columns:
                convs.extend(df[col].dropna().tolist())
        convs = list(set(convs))
        random.shuffle(convs)
        result = [c for c in convs if 50 < len(str(c)) < 500][:max_n]
        print(f'  SmileStyle: {len(result)}개')
        return result
    except Exception as e:
        print(f'  SmileStyle 실패: {e}')
        return []

def load_kakao(path='ChatbotData.csv', max_n=300):
    """카카오톡 대화 데이터에서 일상대화 추출"""
    threat_kw = ['죽','때리','돈 내','협박','폭행','살해','칼','총',
                 '가만 안','신고','경찰','강탈','납치']
    try:
        df = pd.read_csv(path)
        if 'label' in df.columns:
            df = df[df['label'] == 0]
        q_col = 'Q' if 'Q' in df.columns else df.columns[0]
        a_col = 'A' if 'A' in df.columns else df.columns[1]
        convs = []
        for _, row in df.iterrows():
            txt = f"{row[q_col]} {row[a_col]}"
            if not any(k in txt for k in threat_kw) and 20 < len(txt) < 500:
                convs.append(txt)
        random.shuffle(convs)
        result = convs[:max_n]
        print(f'  KakaoChat: {len(result)}개')
        return result
    except Exception as e:
        print(f'  KakaoChat 실패: {e}')
        return []

def load_unsmile(max_n=200):
    """kor_unsmile에서 clean(비혐오) 문장 추출"""
    try:
        from datasets import load_dataset
        ds = load_dataset('smilegate-ai/kor_unsmile', split='train')
        df = ds.to_pandas()
        texts = df[df['clean']==1.0]['문장'].tolist()
        filtered = [t for t in texts if 20 < len(str(t)) < 500]
        random.shuffle(filtered)
        result = filtered[:max_n]
        print(f'  kor_unsmile clean: {len(result)}개')
        return result
    except Exception as e:
        print(f'  kor_unsmile 실패: {e}')
        return []

def load_nsmc(max_n=100):
    """NSMC에서 긍정적 일상 표현 추출"""
    try:
        from datasets import load_dataset
        ds = load_dataset('nsmc', split='train')
        df = ds.to_pandas()
        pos = df[df['label']==1]['document'].dropna().tolist()
        daily_kw = ['재밌','좋아','추천','감동','최고','웃기',
                    '행복','사랑','기분','즐거','편하']
        filtered = [t for t in pos
                    if any(k in str(t) for k in daily_kw)
                    and 20 < len(str(t)) < 300]
        random.shuffle(filtered)
        result = filtered[:max_n]
        print(f'  NSMC positive: {len(result)}개')
        return result
    except Exception as e:
        print(f'  NSMC 실패: {e}')
        return []

print('✅ 수집 함수 정의 완료')

In [ ]:
# ═══ 2-2: 경계 케이스 직접 생성 ⭐ ═══
# 위협처럼 보이지만 실제로는 일반대화인 경우
# → 모델이 진짜 위협 vs 농담을 구분하는 핵심 데이터

BOUNDARY_CASES = [
    "야 죽을래 ㅋㅋ 아 진짜 웃겨서 죽겠다 어제 그거 봤어 뭔데 유튜브에 고양이 영상 보고 진짜 빵 터졌어 나도 보여줘 보내줄게",
    "돈 내놔 ㅋㅋ 밥값 네가 쏜다며 지난번에 내가 냈잖아 맞다 그럼 오늘은 내가 낼게 그래 맛있는 거 먹자 삼겹살 어때 좋아",
    "야 너 진짜 때리고 싶다 왜 맨날 늦어 미안 버스 놓쳤어 다음부터 일찍 나와 알겠어 진짜 다음에 또 늦으면 밥 사야 돼",
    "아 미쳐 진짜 죽겠다 시험 망했어 몇 점인데 말하기 싫어 괜찮아 다음에 잘하면 되지 위로해줘서 고마워 커피나 마시러 가자",
    "야 내 돈 내놔 뭐 아까 내가 빌려준 천원 아 맞다 미안 잊어버렸어 괜찮아 다음에 사탕 하나 사줘 ㅋㅋ 알겠어",
    "너 오늘 죽었어 왜 오늘 게임에서 내가 이길 거거든 꿈꿔 ㅋㅋ 내가 더 잘해 한판 하자 콜 지는 사람 아이스크림 사기다",
    "진짜 폭발할 것 같아 왜 무슨 일이야 과제 양이 너무 많아서 미쳐버리겠어 같이 하자 고마워 카페에서 하자 좋아 가자",
    "야 개빡쳐 왜 무슨 일이야 폰 액정 깨졌어 아이고 AS 가봐 비싸겠다 보험 안 들었어 아 그건 좀 아깝다 폰 빌려줄까",
    "너 때문에 피곤해서 죽겠어 왜 어제 전화 3시간 했잖아 ㅋㅋ 재밌었는데 나도 그래도 오늘은 일찍 자자 알겠어 굿나잇",
    "아 열받아 죽겠네 왜 버스 코앞에서 놓쳤어 ㅋㅋ 다음 거 타면 되지 10분 후에 온대 기다리자 편의점 갔다올게 삼각김밥 사와",
    "야 너 나한테 맞을래 뭐 내가 라면 먹는데 안 불러줬잖아 미안 다음에 같이 먹자 진짜지 약속이야 꼭 불러줘",
    "야 너 미쳤어 왜 이 케이크 맛이 미쳤어 진짜 어디서 샀어 역 앞에 새로 생긴 베이커리 나도 가보고 싶다 내일 같이 가자",
    "때려치우고 싶다 뭘 회사 오늘 진짜 힘들었어 무슨 일이야 야근이었어 고생했다 맥주 한잔하자 좋아 치킨도 시키자",
    "살려줘 왜 무슨 일이야 과제 마감이 내일인데 아직 시작도 안 했어 큰일이다 같이 하자 도서관 갈래 응 빨리 가자",
    "죽여준다 진짜 뭐 이 라면 맛이 죽여줘 어디서 샀어 편의점 신상이야 나도 사먹어봐야겠다 강추야 저녁에 먹어볼게",
    "너 내일 각오해 왜 무슨 일이야 내일 볼링 대회잖아 나한테 질 각오하라고 ㅋㅋ 꿈꿔 내가 이길 건데 내기하자",
    "폭탄 맞은 기분이야 뭐 머리를 잘랐는데 너무 짧아 ㅋㅋ 사진 보여줘 와 진짜 짧다 금방 자라 그래도 좀 충격이야",
    "야 거기 서 왜 뒤에서 불러서 놀랐잖아 미안 ㅋㅋ 같이 점심 먹으러 가자 뭐 먹을까 돈까스 어때 좋아 빨리 가자",
    "협박 아닌데 내일 안 오면 진짜 서운해 뭔데 생일파티 하려고 하는데 와야지 몰랐어 당연히 갈게 선물 준비할게 고마워",
    "너 오늘 큰일난다 왜 무슨 일이야 축구 시합인데 상대팀이 강해 이길 수 있을까 연습 열심히 했잖아 화이팅 고마워",
    "칼 같은 사람이야 누가 우리 팀장님 시간 약속 칼 같아 좋은 거 아니야 맞아 배울 점이야 나도 그러고 싶다",
    "야 폭주하고 싶다 왜 시험 끝나서 기분 좋아서 ㅋㅋ 놀러 가자 어디 갈까 놀이공원 가자 좋아 이번 주말 콜",
    "강도 높은 운동했어 뭐 했어 크로스핏 시작했어 힘들지 않아 처음에는 죽을 것 같았어 ㅋㅋ 대단하다 나도 해볼까",
    "전쟁이다 뭐 오늘 세일인데 사람이 너무 많아 뭐 사려고 운동화 나이키 세일이래 나도 가고 싶다 빨리 와 자리 맡아놨어",
    "돈 뺏길 뻔했어 뭐 무슨 일이야 아니 게임에서 사기 당할 뻔 운이 좋았네 맞아 조심해야지 다음부터 확인하고 거래하자",
]

print(f'경계 케이스: {len(BOUNDARY_CASES)}개 생성')

In [ ]:
# ═══ 2-3: 전체 합성데이터 수집 실행 ═══
print('🚀 합성데이터 수집 시작!\n')

src1 = load_smilestyle(max_n=400)
src2 = load_kakao(max_n=300)
src3 = load_unsmile(max_n=200)
src4 = load_nsmc(max_n=100)
src5 = BOUNDARY_CASES

all_normal = list(set(src1 + src2 + src3 + src4 + src5))
random.shuffle(all_normal)

# 길이 필터 (너무 짧거나 긴 것 제거)
all_normal = [c for c in all_normal if 20 < len(str(c)) < 500]

print(f'\n📊 수집 결과:')
print(f'  SmileStyle: {len(src1)}개')
print(f'  KakaoChat:  {len(src2)}개')
print(f'  UnSmile:    {len(src3)}개')
print(f'  NSMC:       {len(src4)}개')
print(f'  경계케이스: {len(src5)}개')
print(f'  ─────────────────────')
print(f'  중복제거 후 총: {len(all_normal)}개')

# DataFrame으로 변환
normal_df = pd.DataFrame({
    'idx': [f'synth_{i:04d}' for i in range(len(all_normal))],
    'class': '일반 대화',
    'conversation': all_normal,
    'label': 4
})

print(f'\n길이 통계: mean={normal_df["conversation"].str.len().mean():.0f}자')

In [ ]:
# ═══ 2-4: 학습 데이터 통합 ═══
train_df['label'] = train_df['class'].map(CLASS_MAP)
normal_df['conv_len'] = normal_df['conversation'].str.len()

train_full = pd.concat([train_df, normal_df], ignore_index=True)

print('합성데이터 추가 후 전체 학습 데이터:')
print(train_full['class'].value_counts())
print(f'\n총: {len(train_full)}개')

---
## 🔧 STEP 3: 텍스트 전처리

**[배운 것 활용]** Ex06 텍스트 정규화, Ex07 형태소 분석 vs 서브워드 토큰화

In [ ]:
# ═══ 3-1: 전처리 함수 ═══
def preprocess(text):
    """줄바꿈→공백, 연속공백 제거, strip"""
    if pd.isna(text): return ''
    text = str(text).replace('\n', ' ')
    text = re.sub(r'\s+', ' ', text).strip()
    return text

train_full['text'] = train_full['conversation'].apply(preprocess)
test_df['text'] = test_df['conversation'].apply(preprocess)

print('전처리 완료')
print(f'  전: {train_full["conversation"].iloc[0][:80]}...')
print(f'  후: {train_full["text"].iloc[0][:80]}...')

---
## 🧠 STEP 4: 모델 & 토크나이저 + Dataset 정의

**[모델 선정 근거] KcELECTRA**
1. 한국어 댓글/구어체 사전학습 → DKTC 도메인과 유사
2. ELECTRA RTD 방식: 모든 토큰을 학습에 활용 → 소규모 데이터에 효율적
3. BERT MLM 대비 동일 계산량에서 성능 우위 (Clark et al., 2020)

**[배운 것 활용]** Ex09 Transformer 구현 → 사전학습 모델 Fine-tuning

In [ ]:
# ═══ 4-1: 토크나이저 로드 & 토큰 길이 분석 ═══
MODEL_NAME = 'beomi/KcELECTRA-base-v2022'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# MAX_LEN 결정 근거
token_lengths = [len(tokenizer.tokenize(t)) for t in train_full['text']]
print(f'토큰 길이 분포:')
print(f'  평균: {np.mean(token_lengths):.0f}')
print(f'  90%ile: {np.percentile(token_lengths, 90):.0f}')
print(f'  95%ile: {np.percentile(token_lengths, 95):.0f}')
print(f'  → MAX_LEN={MAX_LEN}으로 95%+ 커버')

In [ ]:
# ═══ 4-2: Dataset 클래스 정의 ═══
# [Ex06 활용] NSMCDataset과 동일 구조

class DKTCDataset(Dataset):
    def __init__(self, texts, labels=None, tokenizer=None, max_len=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = self.tokenizer(
            str(self.texts[idx]), max_length=self.max_len,
            padding='max_length', truncation=True, return_tensors='pt'
        )
        item = {k: v.squeeze() for k, v in enc.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

print('✅ DKTCDataset 정의 완료')

In [ ]:
# ═══ 4-3: Loss 함수 정의 ═══

class FocalLoss(nn.Module):
    """
    Focal Loss (Lin et al., 2017)
    FL(p_t) = -alpha * (1 - p_t)^gamma * log(p_t)
    → 쉬운 샘플의 loss를 줄이고, 어려운 샘플에 집중
    """
    def __init__(self, alpha=None, gamma=2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        ce = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce)
        loss = (1 - pt) ** self.gamma * ce
        if self.alpha is not None:
            loss = self.alpha[targets] * loss
        return loss.mean()

def rdrop_loss(logits1, logits2, labels, criterion, alpha=0.7):
    """
    R-Drop (Liang et al., 2021)
    같은 입력 2번 forward → 두 출력의 KL divergence 최소화
    → 드롭아웃 랜덤성을 활용한 정규화
    """
    ce = (criterion(logits1, labels) + criterion(logits2, labels)) / 2
    p1 = F.log_softmax(logits1, dim=-1)
    p2 = F.log_softmax(logits2, dim=-1)
    q1 = F.softmax(logits1, dim=-1)
    q2 = F.softmax(logits2, dim=-1)
    kl = (F.kl_div(p1, q2, reduction='batchmean') +
          F.kl_div(p2, q1, reduction='batchmean')) / 2
    return ce + alpha * kl

print('✅ FocalLoss, R-Drop 정의 완료')

---
## 🔥 STEP 5: 학습 + Ablation Study

**[배운 것 활용]** Ex03 Ablation Study 방법론

| 실험 | 설정 | 비교 포인트 |
|---|---|---|
| Exp1 | CE Loss (baseline) | 기준점 |
| Exp2 | + Focal Loss | 클래스 불균형 해결 효과 |
| Exp3 | + Focal + R-Drop | 과적합 방지 효과 |
| Exp4 | 합성 500개 축소 | 데이터 양의 영향 |

In [ ]:
# ═══ 5-1: 실험 실행 함수 ═══

def run_experiment(
    name, train_data, model_name=MODEL_NAME,
    epochs=5, lr=2e-5,
    use_focal=False, use_rdrop=False, rdrop_alpha=0.7
):
    set_seed(42)
    print(f'\n{"─"*60}')
    print(f'🧪 {name}')
    print(f'   Train: {len(train_data)}, Focal: {use_focal}, R-Drop: {use_rdrop}')
    print(f'{"─"*60}')

    tok = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name, num_labels=NUM_CLASSES
    ).to(device)

    texts = train_data['text'].values
    labels = train_data['label'].values
    X_tr, X_val, y_tr, y_val = train_test_split(
        texts, labels, test_size=0.15, random_state=42, stratify=labels
    )

    tr_loader = DataLoader(
        DKTCDataset(X_tr, y_tr, tok, MAX_LEN),
        batch_size=BATCH_SIZE, shuffle=True
    )
    va_loader = DataLoader(
        DKTCDataset(X_val, y_val, tok, MAX_LEN),
        batch_size=BATCH_SIZE
    )

    # Loss
    if use_focal:
        counts = np.bincount(labels, minlength=NUM_CLASSES).astype(float)
        w = 1.0 / np.maximum(counts, 1)
        w = w / w.sum() * NUM_CLASSES
        criterion = FocalLoss(alpha=torch.FloatTensor(w).to(device), gamma=2.0)
    else:
        criterion = nn.CrossEntropyLoss()

    optimizer = AdamW(model.parameters(), lr=lr, weight_decay=0.01)
    total_steps = len(tr_loader) * epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=int(total_steps*0.1),
        num_training_steps=total_steps
    )

    best_f1 = 0
    history = {'train_loss':[], 'val_f1':[]}

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch in tr_loader:
            ids = batch['input_ids'].to(device)
            mask = batch['attention_mask'].to(device)
            lbl = batch['labels'].to(device)
            optimizer.zero_grad()

            if use_rdrop:
                o1 = model(input_ids=ids, attention_mask=mask)
                o2 = model(input_ids=ids, attention_mask=mask)
                loss = rdrop_loss(o1.logits, o2.logits, lbl, criterion, rdrop_alpha)
            else:
                o = model(input_ids=ids, attention_mask=mask)
                loss = criterion(o.logits, lbl)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            total_loss += loss.item()

        # Validation
        model.eval()
        preds, lbls = [], []
        with torch.no_grad():
            for batch in va_loader:
                ids = batch['input_ids'].to(device)
                mask = batch['attention_mask'].to(device)
                lbl = batch['labels'].to(device)
                o = model(input_ids=ids, attention_mask=mask)
                preds.extend(torch.argmax(o.logits, dim=-1).cpu().numpy())
                lbls.extend(lbl.cpu().numpy())

        f1 = f1_score(lbls, preds, average='macro')
        tl = total_loss / len(tr_loader)
        history['train_loss'].append(tl)
        history['val_f1'].append(f1)

        star = ''
        if f1 > best_f1:
            best_f1 = f1
            torch.save(model.state_dict(), f'best_{name}.pt')
            best_preds, best_lbls = preds, lbls
            star = ' ⭐'

        print(f'   Ep {epoch+1}/{epochs} | Loss: {tl:.4f} | Val F1: {f1:.4f}{star}')

    # 최종 리포트
    model.load_state_dict(torch.load(f'best_{name}.pt'))
    print(f'\n   🏆 Best Val F1: {best_f1:.4f}')
    print(classification_report(best_lbls, best_preds, target_names=CLASS_NAMES))

    return {
        'name': name, 'val_f1': best_f1,
        'history': history,
        'model': model, 'tokenizer': tok,
        'preds': best_preds, 'labels': best_lbls
    }

print('✅ run_experiment 함수 정의 완료')

In [ ]:
# ═══ 5-2: Exp1 - Baseline (CE Loss) ═══
r1 = run_experiment('Exp1_CE_baseline', train_full, use_focal=False, use_rdrop=False)

In [ ]:
# ═══ 5-3: Exp2 - Focal Loss ═══
r2 = run_experiment('Exp2_FocalLoss', train_full, use_focal=True, use_rdrop=False)

In [ ]:
# ═══ 5-4: Exp3 - Focal Loss + R-Drop ═══
r3 = run_experiment('Exp3_Focal_RDrop', train_full, use_focal=True, use_rdrop=True)

In [ ]:
# ═══ 5-5: Exp4 - 합성데이터 500개로 축소 ═══
n500 = min(500, len(normal_df))
normal_500 = normal_df.sample(n=n500, random_state=42)
train_500 = pd.concat([
    train_df[['text','label']],
    normal_500[['text','label']]
], ignore_index=True)

# text 컬럼이 없을 수 있으므로 보정
if 'text' not in train_df.columns:
    train_df['text'] = train_df['conversation'].apply(preprocess)
if 'text' not in normal_500.columns:
    normal_500['text'] = normal_500['conversation'].apply(preprocess)

train_500 = pd.concat([
    train_df[['text','label']],
    normal_500[['text','label']]
], ignore_index=True)

r4 = run_experiment('Exp4_normal500', train_500, use_focal=True, use_rdrop=True)

---
## 📈 STEP 6: Ablation 결과 시각화

In [ ]:
# ═══ 6-1: Ablation 결과 요약 테이블 ═══
results = [r1, r2, r3, r4]
baseline_f1 = r1['val_f1']

print('=' * 60)
print('📋 Ablation Study 결과 요약')
print('=' * 60)
print(f'{"실험":<25} {"Val F1":>8} {"변화":>8}')
print('─' * 45)
for r in results:
    diff = r['val_f1'] - baseline_f1
    d = f'+{diff:.4f}' if diff > 0 else (f'{diff:.4f}' if diff != 0 else '  base')
    print(f'{r["name"]:<25} {r["val_f1"]:>8.4f} {d:>8}')

best = max(results, key=lambda x: x['val_f1'])
print(f'\n🏆 Best: {best["name"]} (F1: {best["val_f1"]:.4f})')

In [ ]:
# ═══ 6-2: 학습 곡선 비교 그래프 ═══
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

colors = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12']
for i, r in enumerate(results):
    axes[0].plot(r['history']['train_loss'], f'-o', color=colors[i], label=r['name'], markersize=4)
    axes[1].plot(r['history']['val_f1'], f'-o', color=colors[i], label=r['name'], markersize=4)

axes[0].set_title('Train Loss', fontsize=14)
axes[0].set_xlabel('Epoch')
axes[0].legend(fontsize=8)
axes[0].grid(True, alpha=0.3)

axes[1].set_title('Validation F1 Score (Macro)', fontsize=14)
axes[1].set_xlabel('Epoch')
axes[1].legend(fontsize=8)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('ablation_curves.png', dpi=150)
plt.show()

In [ ]:
# ═══ 6-3: Best 모델 Confusion Matrix ═══
cm = confusion_matrix(best['labels'], best['preds'])

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES)
plt.title(f'Confusion Matrix - {best["name"]} (F1: {best["val_f1"]:.4f})', fontsize=13)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.xticks(rotation=45, ha='right', fontsize=8)
plt.yticks(fontsize=8)
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=150)
plt.show()

---
## 🎯 STEP 7: 테스트 예측 & 제출 파일 생성

In [ ]:
# ═══ 7-1: Best 모델로 Test 예측 ═══
best_model = best['model']
best_tok = best['tokenizer']

test_loader = DataLoader(
    DKTCDataset(test_df['text'].values, tokenizer=best_tok, max_len=MAX_LEN),
    batch_size=BATCH_SIZE
)

best_model.eval()
test_preds = []
with torch.no_grad():
    for batch in test_loader:
        ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        o = best_model(input_ids=ids, attention_mask=mask)
        test_preds.extend(torch.argmax(o.logits, dim=-1).cpu().numpy())

print('예측 분포:')
for i, name in enumerate(CLASS_NAMES):
    print(f'  {name}: {test_preds.count(i)}개')

In [ ]:
# ═══ 7-2: submission.csv 생성 ═══
submission_df = pd.read_csv('submission.csv')
submission_df['class'] = test_preds
submission_df.to_csv('submission_final.csv', index=False)

print('✅ submission_final.csv 저장 완료!')
print(submission_df.head(10))

In [ ]:
# ═══ 7-3: 제출 파일 다운로드 (Colab) ═══
from google.colab import files
files.download('submission_final.csv')
print('📥 다운로드 완료! Kaggle에 제출하세요.')

---
## 🎓 STEP 8: 프로젝트 정리 & 발표 포인트

In [ ]:
# ═══ 8-1: 최종 정리 ═══
print(''
'''
╔══════════════════════════════════════════════════════════╗
║           DKTC 프로젝트 최종 정리                        ║
╠══════════════════════════════════════════════════════════╣
║                                                          ║
║  [배운 것 활용]                                          ║
║  ✓ EDA & 전처리: Ex06 텍스트 분석                        ║
║  ✓ 토큰화: Ex07 SentencePiece → ELECTRA WordPiece       ║
║  ✓ 모델: Ex09 Transformer → Fine-tuning                 ║
║  ✓ Ablation: Ex03 실험 비교 방법론                       ║
║                                                          ║
║  [최신 논문 기법]                                        ║
║  ✓ KcELECTRA: 한국어 구어체 특화 (Clark 2020)           ║
║  ✓ Focal Loss: 불균형 해결 (Lin 2017)                   ║
║  ✓ R-Drop: 과적합 방지 (Liang 2021)                     ║
║  ✓ 합성데이터: 5개 소스 통합 (AugGPT 영감)              ║
║                                                          ║
║  [탑다운 접근법]                                         ║
║  먼저 돌려보고 → 문제 발견 → 최신 기법으로 해결          ║
║                                                          ║
╚══════════════════════════════════════════════════════════╝
''')

# Ablation 결과 재출력
print('\n📋 Ablation Study 최종 결과:')
for r in results:
    diff = r['val_f1'] - baseline_f1
    d = f'+{diff:.4f}' if diff > 0 else (f'{diff:.4f}' if diff != 0 else '  base')
    print(f'  {r["name"]:<25} F1: {r["val_f1"]:.4f} ({d})')

print(f'\n🏆 Best: {best["name"]} → F1: {best["val_f1"]:.4f}')
print('\n🎉 프로젝트 완료!')